In [ ]:
import requests
import pandas as pd
import numpy as np
import re
import json
from pandas import json_normalize

In [ ]:
pd.options.display.max_columns = None
pd.options.display.max_rows = None
pd.set_option('precision', 7)

## Getting Families Coordinates

This code uses 'FamlyName_withCoords.csv'
Coordinates are for the family farms, Poso de means water well

In [ ]:
cd sample_data

/content/sample_data


In [ ]:
families_coordinates = pd.read_csv('FamilyName_withCoords.csv', index_col=0)
families_coordinates.head(2)

,Name,geometry
0,Poso de Rebeca Sequeira,POINT (-86.95236730000001 12.4635646)
1,Poso de Fátima Castillo,POINT (-86.9832019 12.4433067)


In [ ]:
# New column coordinate made from splitting geometry
families_coordinates['coordinate'] = families_coordinates['geometry'].str.extract(r'\(([^)]+)\)')
# Longitude and Latitude Columns made from splitting coordinate
families_coordinates[['longitude','latitude']] = families_coordinates['coordinate'].str.split(pat=' ', expand=True)
# families_coordinates['Name'] = families_coordinates['Name'].str[8:]

# Make Latitude and Longitude columns type numeric 
#and to 7 decimal places for api requests
families_coordinates['longitude'] = pd.to_numeric(families_coordinates['longitude'])
families_coordinates['longitude'] = families_coordinates['longitude'].round(decimals=7)
families_coordinates['latitude'] = pd.to_numeric(families_coordinates['latitude'])
families_coordinates['latitude'] = families_coordinates['latitude'].round(7)

# Columns are reordered for latitude, longitude format
families_coordinates = families_coordinates[['Name', 'geometry', 'latitude', 'longitude', 'coordinate']]
families_coordinates.head(1)

,Name,geometry,latitude,longitude,coordinate
0,Poso de Rebeca Sequeira,POINT (-86.95236730000001 12.4635646),12.4635646,-86.9523673,-86.95236730000001 12.4635646


In [ ]:
Making Weather Call Lists

In [ ]:
### Making lists for API Weather calls
names = families_coordinates['Name'].tolist()

# Formatting latitudes to 7 decimal format
latitudes = families_coordinates['latitude'].tolist()
latitudes2 = list()
for lat in latitudes:
    lat = format(lat, '.7f')
    latitudes2.append(lat)
latitudes = latitudes2

# Formatting longitudes to 7 decimal format
longitudes = families_coordinates['longitude'].tolist()
longitudes2 = list()
for lon in longitudes:
    lon = format(lon, '.7f')
    longitudes2.append(lon)
longitudes = longitudes2

## Calling Weather API

In [ ]:
# Setting open weather api 
coord_API_endpoint = "http://api.openweathermap.org/data/2.5/weather?"
apikey = '6e8159e0b50a4806600492aa4ffd31fe'
join_key = "&appid=" + apikey
units = "&units=imperial"

In [ ]:
# empty list to store each family's JSON Data files within
families_json_dataset = list()

In [ ]:
# API Call for Each Family 
for name, latitude, longitude in zip(names, latitudes, longitudes):
    
    #creating Key
    latlonkey = "lat=" + str(latitude) + "&lon=" + str(longitude)
    current_coord_weather_url = coord_API_endpoint + latlonkey + join_key + units
    print(name, current_coord_weather_url)
    # Gets Data 
    json_data = requests.get(current_coord_weather_url).json()
    # Into JSON file list
    families_json_dataset.append(json_data)

Poso de Rebeca Sequeira http://api.openweathermap.org/data/2.5/weather?lat=12.4635646&lon=-86.9523673&appid=6e8159e0b50a4806600492aa4ffd31fe&units=imperial
Poso de Fátima Castillo http://api.openweathermap.org/data/2.5/weather?lat=12.4433067&lon=-86.9832019&appid=6e8159e0b50a4806600492aa4ffd31fe&units=imperial
Poso de Yojhana  Cristina Flores http://api.openweathermap.org/data/2.5/weather?lat=12.4609104&lon=-86.9963934&appid=6e8159e0b50a4806600492aa4ffd31fe&units=imperial
Poso de Karla Galeano http://api.openweathermap.org/data/2.5/weather?lat=12.4551053&lon=-86.9968179&appid=6e8159e0b50a4806600492aa4ffd31fe&units=imperial
Poso de Rita Arevalo http://api.openweathermap.org/data/2.5/weather?lat=12.4543169&lon=-86.9966864&appid=6e8159e0b50a4806600492aa4ffd31fe&units=imperial
Poso de Cristina Alvares http://api.openweathermap.org/data/2.5/weather?lat=12.4516219&lon=-86.9985749&appid=6e8159e0b50a4806600492aa4ffd31fe&units=imperial
Poso de Luz Marina Rivas http://api.openweathermap.org/data

In [ ]:
{'base': 'stations',
  'clouds': {'all': 40},
  'cod': 200,
  'coord': {'lat': 12.4636, 'lon': -86.9524},
  'dt': 1615500787,
  'id': 3617115,
  'main': {'feels_like': 96.19,
          'humidity': 49,
          'pressure': 1010,
          'temp': 91.4,
          'temp_max': 91.4,
          'temp_min': 91.4},
  'name': 'Quezalguaque',
  'sys': {'country': 'NI',
          'id': 7172,
          'sunrise': 1615463853,
          'sunset': 1615507087,
          'type': 1},
  'timezone': -21600,
  'visibility': 10000,
  'weather': [{'description': 'scattered clouds',
              'icon': '03d',
              'id': 802,
              'main': 'Clouds'}],
  'wind': {'deg': 190, 'speed': 4.61}}

In [ ]:
pd.json_normalize(families_json_dataset, record_path=['weather'],
                  meta=['clouds'])

,id,main,description,icon,clouds
0,802,Clouds,scattered clouds,03d,{'all': 40}
1,802,Clouds,scattered clouds,03d,{'all': 40}
2,802,Clouds,scattered clouds,03d,{'all': 40}
3,802,Clouds,scattered clouds,03d,{'all': 40}
4,802,Clouds,scattered clouds,03d,{'all': 40}
5,802,Clouds,scattered clouds,03d,{'all': 40}
6,802,Clouds,scattered clouds,03d,{'all': 40}
7,802,Clouds,scattered clouds,03d,{'all': 40}
8,802,Clouds,scattered clouds,03d,{'all': 40}
9,802,Clouds,scattered clouds,03d,{'all': 40}


In [ ]:
families_json_dataset

[{'base': 'stations',
  'clouds': {'all': 40},
  'cod': 200,
  'coord': {'lat': 12.4636, 'lon': -86.9524},
  'dt': 1615500787,
  'id': 3617115,
  'main': {'feels_like': 96.19,
   'humidity': 49,
   'pressure': 1010,
   'temp': 91.4,
   'temp_max': 91.4,
   'temp_min': 91.4},
  'name': 'Quezalguaque',
  'sys': {'country': 'NI',
   'id': 7172,
   'sunrise': 1615463853,
   'sunset': 1615507087,
   'type': 1},
  'timezone': -21600,
  'visibility': 10000,
  'weather': [{'description': 'scattered clouds',
    'icon': '03d',
    'id': 802,
    'main': 'Clouds'}],
  'wind': {'deg': 190, 'speed': 4.61}},
 {'base': 'stations',
  'clouds': {'all': 40},
  'cod': 200,
  'coord': {'lat': 12.4433, 'lon': -86.9832},
  'dt': 1615500788,
  'id': 3617115,
  'main': {'feels_like': 96.19,
   'humidity': 49,
   'pressure': 1010,
   'temp': 91.4,
   'temp_max': 91.4,
   'temp_min': 91.4},
  'name': 'Quezalguaque',
  'sys': {'country': 'NI',
   'id': 7172,
   'sunrise': 1615463861,
   'sunset': 1615507094,
  

In [ ]:
# last family data call
json_data

{'base': 'stations',
 'clouds': {'all': 20},
 'cod': 200,
 'coord': {'lat': 12.4609, 'lon': -86.9997},
 'dt': 1615502390,
 'id': 3617191,
 'main': {'feels_like': 97.97,
  'humidity': 55,
  'pressure': 1009,
  'temp': 91.4,
  'temp_max': 91.4,
  'temp_min': 91.4},
 'name': 'Posoltega',
 'sys': {'country': 'NI',
  'id': 7172,
  'sunrise': 1615463865,
  'sunset': 1615507098,
  'type': 1},
 'timezone': -21600,
 'visibility': 10000,
 'weather': [{'description': 'few clouds',
   'icon': '02d',
   'id': 801,
   'main': 'Clouds'}],
 'wind': {'deg': 190, 'speed': 4.61}}

#### Storing

In [ ]:
df_all_current_weather = pd.DataFrame()
prediction_num = 0

# Create empty lists to store the JSON Data
current_weather_id = []
current_time = []
own_city_id = []
city = []
latitude = []
longitude = []
country = []
timezone = []
sunrise = []
sunset = []
temperature = []
temperature_feel = []
temperature_min = []
temperature_max = []
pressure = []
humidity = []
main = []
main_description = []
clouds = []
wind_speed = []
wind_degree = []
visibility = []

In [ ]:
# Add JSON Data to the lists
for each in families_json_dataset:
  prediction_num +=1
  current_weather_id.append(prediction_num + 1)
  current_time.append(pd.Timestamp.now())
  own_city_id.append(json_data['id'])
  city.append(json_data['name'])
  latitude.append(json_data['coord']['lat'])
  longitude.append(json_data['coord']['lon'])
  country.append(json_data['sys']['country'])
  if json_data['timezone'] >0 :
      timezone.append(("+" + str((json_data['timezone'])/3600)))
  else:
      timezone.append(((json_data['timezone'])/3600))
  sunrise.append(json_data['sys']['sunrise'])
  sunset.append(json_data['sys']['sunset'])
  temperature.append(json_data['main']['temp'])
  temperature_feel.append(json_data['main']['feels_like'])
  temperature_min.append(json_data['main']['temp_min'])
  temperature_max.append(json_data['main']['temp_max'])
  pressure.append(json_data['main']['pressure'])
  humidity.append(json_data['main']['humidity'])
  main.append(json_data['weather'][0]['main'])
  main_description.append(json_data['weather'][0]['description'])
  clouds.append(json_data['clouds']['all'])
  wind_speed.append(json_data['wind']['speed'])
  wind_degree.append(json_data['wind']['deg'])
  visibility.append(json_data['visibility'])

In [ ]:
# Write Lists to DataFrame
df_all_current_weather = pd.DataFrame()
# df_all_current_weather['current_weather_id'] = current_weather_id
df_all_current_weather['current_time'] = current_time
df_all_current_weather['own_city_id'] = own_city_id
df_all_current_weather['city'] = city
df_all_current_weather['latitude'] = latitude
df_all_current_weather['longitude'] = longitude
df_all_current_weather['country'] = country
df_all_current_weather['timezone'] = timezone
df_all_current_weather['sunrise'] = sunrise
df_all_current_weather['sunset'] = sunset
df_all_current_weather['temperature'] = temperature
df_all_current_weather['temperature_feel'] = temperature_feel
df_all_current_weather['temperature_min'] = temperature_min
df_all_current_weather['temperature_max'] = temperature_max
df_all_current_weather['pressure'] = pressure
df_all_current_weather['humidity'] = humidity
df_all_current_weather['main'] = main
df_all_current_weather['main_description'] = main_description
df_all_current_weather['clouds'] = clouds
df_all_current_weather['wind_speed'] = wind_speed
df_all_current_weather['wind_degree'] = wind_degree
df_all_current_weather['visibility'] = visibility

In [ ]:
df_all_current_weather

,current_time,own_city_id,city,latitude,longitude,country,timezone,sunrise,sunset,temperature,temperature_feel,temperature_min,temperature_max,pressure,humidity,main,main_description,clouds,wind_speed,wind_degree,visibility
0,2021-03-11 22:43:01.051096,3617191,Posoltega,12.4609,-86.9997,NI,-6.0,1615463865,1615507098,91.4,97.97,91.4,91.4,1009,55,Clouds,few clouds,20,4.61,190,10000
1,2021-03-11 22:43:01.051226,3617191,Posoltega,12.4609,-86.9997,NI,-6.0,1615463865,1615507098,91.4,97.97,91.4,91.4,1009,55,Clouds,few clouds,20,4.61,190,10000
2,2021-03-11 22:43:01.051242,3617191,Posoltega,12.4609,-86.9997,NI,-6.0,1615463865,1615507098,91.4,97.97,91.4,91.4,1009,55,Clouds,few clouds,20,4.61,190,10000
3,2021-03-11 22:43:01.051253,3617191,Posoltega,12.4609,-86.9997,NI,-6.0,1615463865,1615507098,91.4,97.97,91.4,91.4,1009,55,Clouds,few clouds,20,4.61,190,10000
4,2021-03-11 22:43:01.051265,3617191,Posoltega,12.4609,-86.9997,NI,-6.0,1615463865,1615507098,91.4,97.97,91.4,91.4,1009,55,Clouds,few clouds,20,4.61,190,10000
5,2021-03-11 22:43:01.051277,3617191,Posoltega,12.4609,-86.9997,NI,-6.0,1615463865,1615507098,91.4,97.97,91.4,91.4,1009,55,Clouds,few clouds,20,4.61,190,10000
6,2021-03-11 22:43:01.051287,3617191,Posoltega,12.4609,-86.9997,NI,-6.0,1615463865,1615507098,91.4,97.97,91.4,91.4,1009,55,Clouds,few clouds,20,4.61,190,10000
7,2021-03-11 22:43:01.051297,3617191,Posoltega,12.4609,-86.9997,NI,-6.0,1615463865,1615507098,91.4,97.97,91.4,91.4,1009,55,Clouds,few clouds,20,4.61,190,10000
8,2021-03-11 22:43:01.051306,3617191,Posoltega,12.4609,-86.9997,NI,-6.0,1615463865,1615507098,91.4,97.97,91.4,91.4,1009,55,Clouds,few clouds,20,4.61,190,10000
9,2021-03-11 22:43:01.051320,3617191,Posoltega,12.4609,-86.9997,NI,-6.0,1615463865,1615507098,91.4,97.97,91.4,91.4,1009,55,Clouds,few clouds,20,4.61,190,10000


In [ ]:
normalized_current_families_weather = json_normalize(families_json_dataset)
normalized_current_families_weather.head(2)

,weather,base,visibility,dt,timezone,id,name,cod,coord.lon,coord.lat,main.temp,main.feels_like,main.temp_min,main.temp_max,main.pressure,main.humidity,wind.speed,wind.deg,clouds.all,sys.type,sys.id,sys.country,sys.sunrise,sys.sunset
0,"[{'id': 802, 'main': 'Clouds', 'description': ...",stations,10000,1615500787,-21600,3617115,Quezalguaque,200,-86.9524,12.4636,91.4,96.19,91.4,91.4,1010,49,4.61,190,40,1,7172,NI,1615463853,1615507087
1,"[{'id': 802, 'main': 'Clouds', 'description': ...",stations,10000,1615500788,-21600,3617115,Quezalguaque,200,-86.9832,12.4433,91.4,96.19,91.4,91.4,1010,49,4.61,190,40,1,7172,NI,1615463861,1615507094


###### Resources Used:

- Gathering Weather Data from Open Weather api [on Medium](https://medium.com/@joseph.magiya/weather-data-and-forecasts-from-open-weather-api-1636691d5ba)

- Breaking Up Columns [With Regular Expressions](https://chrisalbon.com/python/data_wrangling/pandas_regex_to_create_columns/)

- Visualizing Temperature Data over Map 
    - [Google Search](https://www.google.com/search?client=firefox-b-1-d&sxsrf=ALeKk00YYoKZm1iH2M9IdDELIMCdo4Wa6Q%3A1614874311476&ei=xwZBYOnJHMzJ0PEPheeSqAo&q=python+visualization+of+gps+coordinates+and+temperatures&oq=python+visualization+of+gps+coordinates+and+temperatures&gs_lcp=Cgdnd3Mtd2l6EAM6BwgAEEcQsAM6BwgjELACECdQ0xdY7hxg2R5oAXACeACAAYgBiAGgBpIBAzAuN5gBAKABAaoBB2d3cy13aXrIAQjAAQE&sclient=gws-wiz&ved=0ahUKEwipycTNg5fvAhXMJDQIHYWzBKUQ4dUDCAw&uact=5)
    - [Geo Data Base Map](https://jakevdp.github.io/PythonDataScienceHandbook/04.13-geographic-data-with-basemap.html)
    - [Heatmap in Python](https://www.storybench.org/how-to-build-a-heatmap-in-python/)

In [ ]:
df['sys.sunrise'].unique()

array([1614859305, 1614859312, 1614859316, 1614859306, 1614859304,
       1614859308, 1614859302, 1614859313, 1614859317])

# Historic Weather Collection

In [1]:
!pip install wwo-hist

In [2]:
from wwo_hist import retrieve_hist_data
import pandas as pd
import numpy as np

In [3]:
cd sample_data

/content/sample_data


In [4]:
families_coordinates = pd.read_csv('FamilyName_withCoords.csv', index_col=0)

In [7]:
families_coordinates['coordinate'] = families_coordinates['geometry'].str.extract(r'\(([^)]+)\)')
families_coordinates[['longitude','latitude']] = families_coordinates['coordinate'].str.split(pat=' ', expand=True)
# families_coordinates['Name'] = families_coordinates['Name'].str[8:]
families_coordinates['longitude'] = pd.to_numeric(families_coordinates['longitude'])
families_coordinates['longitude'] = families_coordinates['longitude'].round(decimals=2)
families_coordinates['latitude'] = pd.to_numeric(families_coordinates['latitude'])
families_coordinates['latitude'] = families_coordinates['latitude'].round(2)

In [8]:
coordinates = families_coordinates[['Name','latitude','longitude']]
coordinates['apienter'] =  coordinates.latitude.astype(str) + '%' + coordinates.longitude.astype(str)
print(coordinates.apienter.nunique())
coordinates.head(1)

12


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,Name,latitude,longitude,apienter
0,Poso de Rebeca Sequeira,12.46,-86.95,12.46%-86.95


In [9]:
coordinates.to_csv('family_coordinates_api', index=False)

In [19]:
location_list = coordinates['apienter'].unique().tolist()

In [14]:
location_list[:2]

['12.46%-86.95', '12.44%-86.98']

In [12]:
api_key = '064633d6dd5f44e6b4003327210503'

In [20]:
frequency=24
start_date = '1-JAN-2016'
end_date = '9-MAR-2021'
location_list = location_list

In [21]:
vl_hist_weather_data = retrieve_hist_data(api_key,
                                location_list,
                                start_date,
                                end_date,
                                frequency,
                                location_label = False,
                                export_csv = True,
                                store_df = True)



Retrieving weather data for 12.46%-86.95


Currently retrieving data for 12.46%-86.95: from 2016-01-01 to 2016-01-31
Time elapsed (hh:mm:ss.ms) 0:00:01.212520
Currently retrieving data for 12.46%-86.95: from 2016-02-01 to 2016-02-29
Time elapsed (hh:mm:ss.ms) 0:00:02.455469
Currently retrieving data for 12.46%-86.95: from 2016-03-01 to 2016-03-31
Time elapsed (hh:mm:ss.ms) 0:00:03.603195
Currently retrieving data for 12.46%-86.95: from 2016-04-01 to 2016-04-30
Time elapsed (hh:mm:ss.ms) 0:00:04.848631
Currently retrieving data for 12.46%-86.95: from 2016-05-01 to 2016-05-31
Time elapsed (hh:mm:ss.ms) 0:00:05.952878
Currently retrieving data for 12.46%-86.95: from 2016-06-01 to 2016-06-30
Time elapsed (hh:mm:ss.ms) 0:00:08.117105
Currently retrieving data for 12.46%-86.95: from 2016-07-01 to 2016-07-31
Time elapsed (hh:mm:ss.ms) 0:00:09.329299
Currently retrieving data for 12.46%-86.95: from 2016-08-01 to 2016-08-31
Time elapsed (hh:mm:ss.ms) 0:00:10.561976
Currently retrieving data f

In [23]:
type(vl_hist_weather_data)

list

In [30]:
len(vl_hist_weather_data)

12

In [31]:
historical_data = pd.concat(vl_hist_weather_data)

In [41]:
historical_data.to_csv('vl_historical_data.csv', index=False)